In [1]:
# Importacion de librerias

import pathlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn import preprocessing
import math
from tensorflow.python.ops import math_ops
import datetime
import random
from keras.callbacks import EarlyStopping
import scipy.stats as stats
from sklearn.model_selection import StratifiedShuffleSplit


In [ ]:
# Se importa el conjunto de datos 

events_container_path= "C:\\TESIS\\FINAL\\base_datos\\"

dataset1 = pd.read_pickle(events_container_path + "rcc_earthquake_database_2011_2022_v5_1.pkl")      
dataset2 = pd.read_pickle(events_container_path + "rcc_earthquake_database_2023_v5_1.pkl")  


In [ ]:
print(dataset1.shape)
print(dataset2.shape)

In [ ]:
#Concatenamos los datasets
dataset = pd.concat([dataset1, dataset2])
dataset = dataset.reset_index(drop=True)
dataset.shape

In [ ]:
dataset

In [ ]:
# En los datos existen indistintamente  Azimuths nombrados como  0 y 360 se unifican como 0

dataset['Azimuth']= dataset['Azimuth'].replace(360,0)

In [ ]:
# Se eliminan filas que no cumplen con el tiempo de la ventana de datos

count = 0
max_window = 501
to_delete=[]
initial_length= len(dataset)
print ('Tamaño inicial :', initial_length)
for i in range(initial_length):
  tmp=dataset.iloc[i]
  z= tmp['Z_data']
  n= tmp['N_data']
  e= tmp['E_data'] 
  if (len(z) != max_window) or (len(n) != max_window) or (len(e) != max_window):   
    to_delete.append(i)
    count+=1
print('to_delete rows',to_delete)
dataset.drop(to_delete, axis=0, inplace=True)
print ('Eliminadas ',count,' filas')
print(len(dataset))
dataset.shape

In [ ]:
#Restablezco los indices del dataset eliminando las discontinuidades por las filas eliminadas
dataset = dataset.reset_index(drop=True)
dataset

In [ ]:
# Buscamos el maximo valor absoluto de los datos porque existe un dato con un valor maximo erroneo
n =0
global_max = float('-inf')

for i in range(len(dataset)):
    row = dataset.iloc[i]  # Obtén la fila actual
    
    for channel in ['Z_data', 'N_data', 'E_data']:
        channel_values = row[channel]  # Obtén el arreglo correspondiente al canal actual
        
        channel_max = np.max(np.abs(channel_values))  # Encuentra el valor máximo absoluto en el arreglo
        
        # Actualiza el máximo global si es necesario
        if channel_max > global_max:
            global_max = channel_max
            n=i

print("Valor máximo absoluto:", global_max)
print(n)

In [ ]:
#Visualizo el dato erroneo
dataset.iloc[442]['RCC_OBSPY'].plot()

In [ ]:
#Elimino esta fila del dataset que posee problemas  con el valor de los canales y me altera el programa al calcular el maximo
dataset = dataset.drop(dataset.index[442])


In [ ]:
#Restablezco los indices del dataset eliminando las discontinuidades por las filas eliminadas
dataset = dataset.reset_index(drop=True)
dataset

In [ ]:
dataset.info()

In [ ]:
# Se convierte a float las columnas que asi lo requieren

dataset['Mc']= dataset['Mc'].astype(float, errors = 'raise')
dataset['Azimuth']= dataset['Azimuth'].astype(float, errors = 'raise')

In [ ]:
# Convertimos los grados a radianes
dataset['Azimuth_radianes']= [math.radians(i) for i in dataset['Azimuth']]   
dataset

In [ ]:
# Conservo el dataset original
dataset_original= dataset.copy()

In [ ]:
dataset_original

In [ ]:
# Elimino los eventos a mas de  110Km de distancia (1 grado)

indexNames = dataset[ (dataset['Distance'] > 110)].index
dataset.drop(indexNames , inplace=True)
#sns.pairplot(dataset[['Azimuth','Distance','Mc','Depth']], diag_kind="kde")
dataset.shape

In [ ]:
# Crear nuevos DataFrames para cada condición de SNR
dataset_snr25 = dataset[(dataset['SNR_Z_S'] >= 25) & (dataset['SNR_N_S'] >= 25) & (dataset['SNR_E_S'] >= 25)].copy()

dataset_snr20 = dataset[(dataset['SNR_Z_S'] >= 20) & (dataset['SNR_N_S'] >= 20) & (dataset['SNR_E_S'] >= 20)
                        & (dataset['SNR_Z_S'] < 25) & (dataset['SNR_N_S'] < 25) & (dataset['SNR_E_S'] < 25)].copy()
dataset_snr18 = dataset[(dataset['SNR_Z_S'] >= 18) & (dataset['SNR_N_S'] >= 18) & (dataset['SNR_E_S'] >= 18)
                        & (dataset['SNR_Z_S'] < 20) & (dataset['SNR_N_S'] < 20) & (dataset['SNR_E_S'] < 20)].copy()

dataset_snr15 = dataset[(dataset['SNR_Z_S'] >= 15) & (dataset['SNR_N_S'] >= 15) & (dataset['SNR_E_S'] >= 15)
                        & (dataset['SNR_Z_S'] < 18) & (dataset['SNR_N_S'] < 18) & (dataset['SNR_E_S'] < 18)].copy()

dataset_snr10 = dataset[(dataset['SNR_Z_S'] >= 10) & (dataset['SNR_N_S'] >= 10) & (dataset['SNR_E_S'] >= 10)
                        & (dataset['SNR_Z_S'] < 15) & (dataset['SNR_N_S'] < 15) & (dataset['SNR_E_S'] < 15)].copy()

dataset_snr5 = dataset[(dataset['SNR_Z_S'] >= 5) & (dataset['SNR_N_S'] >= 5) & (dataset['SNR_E_S'] >= 5)
                       & (dataset['SNR_Z_S'] < 10) & (dataset['SNR_N_S'] < 10) & (dataset['SNR_E_S'] < 10)].copy()

dataset_snr0 = dataset[(dataset['SNR_Z_S'] > 0) & (dataset['SNR_N_S'] > 0) & (dataset['SNR_E_S'] > 0)
                       & (dataset['SNR_Z_S'] < 5) & (dataset['SNR_N_S'] < 5) & (dataset['SNR_E_S'] < 5)].copy()

# Filtrar los valores que cumplan alguna de las condiciones anteriores o tengan algún valor None en las columnas 
# para crear el dataset_invalid
dataset_invalid = dataset[~(
    (dataset['SNR_Z_S'] >= 25) & (dataset['SNR_N_S'] >= 25) & (dataset['SNR_E_S'] >= 25) |
    ((dataset['SNR_Z_S'] >= 20) & (dataset['SNR_N_S'] >= 20) & (dataset['SNR_E_S'] >= 20) &
    (dataset['SNR_Z_S'] < 25) & (dataset['SNR_N_S'] < 25) & (dataset['SNR_E_S'] < 25)) |
    ((dataset['SNR_Z_S'] >= 18) & (dataset['SNR_N_S'] >= 18) & (dataset['SNR_E_S'] >= 18) &
    (dataset['SNR_Z_S'] < 20) & (dataset['SNR_N_S'] < 20) & (dataset['SNR_E_S'] < 20)) |    
    ((dataset['SNR_Z_S'] >= 15) & (dataset['SNR_N_S'] >= 15) & (dataset['SNR_E_S'] >= 15) &
    (dataset['SNR_Z_S'] < 18) & (dataset['SNR_N_S'] < 18) & (dataset['SNR_E_S'] < 18)) |
    ((dataset['SNR_Z_S'] >= 10) & (dataset['SNR_N_S'] >= 10) & (dataset['SNR_E_S'] >= 10) &
    (dataset['SNR_Z_S'] < 15) & (dataset['SNR_N_S'] < 15) & (dataset['SNR_E_S'] < 15)) |
    ((dataset['SNR_Z_S'] >= 5) & (dataset['SNR_N_S'] >= 5) & (dataset['SNR_E_S'] >= 5) &
    (dataset['SNR_Z_S'] < 10) & (dataset['SNR_N_S'] < 10) & (dataset['SNR_E_S'] < 10)) |
    ((dataset['SNR_Z_S'] > 0) & (dataset['SNR_N_S'] > 0) & (dataset['SNR_E_S'] > 0) &
    (dataset['SNR_Z_S'] < 5) & (dataset['SNR_N_S'] < 5) & (dataset['SNR_E_S'] < 5))) &
    (dataset['SNR_Z_S'].notna() & dataset['SNR_N_S'].notna() & dataset['SNR_E_S'].notna())].copy()

# Crear el dataset_None con los elementos que tienen algún valor None en las columnas
dataset_None = dataset[dataset['SNR_Z_S'].isna() | dataset['SNR_N_S'].isna() | dataset['SNR_E_S'].isna()].copy()

# Marcar la columna 'Is_Valid' en cada DataFrame
dataset_snr25['Is_Valid'] = True
dataset_snr20['Is_Valid'] = False
dataset_snr18['Is_Valid'] = False
dataset_snr15['Is_Valid'] = False
dataset_snr10['Is_Valid'] = False
dataset_snr5['Is_Valid'] = False
dataset_snr0['Is_Valid'] = False
dataset_invalid['Is_Valid'] = False
dataset_None['Is_Valid'] = False

# Verificar los nuevos DataFrames
print("Dataset SNR >= 25:")
print(dataset_snr25.shape)

print("\nDataset SNR >= 20:")
print(dataset_snr20.shape)

print("\nDataset SNR >= 18:")
print(dataset_snr18.shape)

print("\nDataset SNR >= 15:")
print(dataset_snr15.shape)

print("\nDataset SNR >= 10:")
print(dataset_snr10.shape)

print("\nDataset SNR >= 5:")
print(dataset_snr5.shape)

print("\nDataset SNR > 0:")
print(dataset_snr0.shape)

print("\nDataset Invalid:")
print(dataset_invalid.shape)

print("\nDataset None:")
print(dataset_None.shape)

In [ ]:
#Restablezco los indices del dataset eliminando las discontinuidades por las filas eliminadas
dataset_snr25 = dataset_snr25.reset_index(drop=True)
dataset_snr20 = dataset_snr20.reset_index(drop=True)
dataset_snr18 = dataset_snr18.reset_index(drop=True)
dataset_snr15 = dataset_snr15.reset_index(drop=True)
dataset_snr10 = dataset_snr10.reset_index(drop=True)
dataset_snr5 = dataset_snr5.reset_index(drop=True)
dataset_snr0 = dataset_snr0.reset_index(drop=True)
dataset_invalid = dataset_invalid.reset_index(drop=True)
dataset_None = dataset_None.reset_index(drop=True)

In [ ]:
#Busco en el dataset_invalid aquellos datos que cumplen la SNR por cada canal independiente

In [ ]:
dataset_snr_any_channel = dataset_invalid[((dataset_invalid['SNR_Z_S'] >= 15) & 
                                           ((dataset_invalid['SNR_N_S'] >= 10) &
                                           (dataset_invalid['SNR_E_S'] >= 10)))|
                                           ((dataset_invalid['SNR_N_S'] >= 15) & 
                                           ((dataset_invalid['SNR_Z_S'] >= 10) & 
                                           (dataset_invalid['SNR_E_S'] >= 10)))|
                                           ((dataset_invalid['SNR_E_S'] >= 15) &
                                           ((dataset_invalid['SNR_Z_S'] >= 10) &
                                           (dataset_invalid['SNR_N_S'] >= 10)))].copy()

dataset_new_invalid = dataset_invalid[~(((dataset_invalid['SNR_Z_S'] >= 15) &
                                       ((dataset_invalid['SNR_N_S'] >= 10) &
                                       (dataset_invalid['SNR_E_S'] >= 10)))|
                                       ((dataset_invalid['SNR_N_S'] >= 15) &
                                       ((dataset_invalid['SNR_Z_S'] >= 10) &
                                       (dataset_invalid['SNR_E_S'] >= 10)))|
                                       ((dataset_invalid['SNR_E_S'] >= 15) &
                                       ((dataset_invalid['SNR_Z_S'] >= 10) & (dataset_invalid['SNR_N_S'] >= 10))))].copy()



pd.options.display.max_rows = 20
pd.options.display.max_columns = 999

#Restablezco los indices del dataset eliminando las discontinuidades por las filas eliminadas
dataset_snr_any_channel = dataset_snr_any_channel.reset_index(drop=True)
dataset_new_invalid = dataset_new_invalid.reset_index(drop=True)
# Verificar los nuevos DataFrames
print("Dataset SNR Any Channel :")
print(dataset_snr_any_channel.shape)

print("Dataset SNR New Invalid:")
print(dataset_new_invalid.shape)


In [ ]:
dataset_snr_any_channel

In [ ]:
#Trabajaremos con los dataset_None pero utilizando la SNR de P

In [ ]:
dataset_None

In [ ]:
# Crear nuevos DataFrames None para cada condición
dataset_None_snr25 = dataset_None[(dataset_None['SNR_Z_P'] >= 25) & (dataset_None['SNR_N_P'] >= 25) &
                                  (dataset_None['SNR_E_P'] >= 25)].copy()

dataset_None_snr20 = dataset_None[(dataset_None['SNR_Z_P'] >= 20) & (dataset_None['SNR_N_P'] >= 20) &
                                  (dataset_None['SNR_E_P'] >= 20) &
                                  (dataset_None['SNR_Z_P'] < 25) &
                                  (dataset_None['SNR_N_P'] < 25) & (dataset_None['SNR_E_P'] < 25)].copy()
dataset_None_snr18 = dataset_None[(dataset_None['SNR_Z_P'] >= 18) &
                                  (dataset_None['SNR_N_P'] >= 18) & (dataset_None['SNR_E_P'] >= 18) &
                                  (dataset_None['SNR_Z_P'] < 20) &
                                  (dataset_None['SNR_N_P'] < 20) &
                                  (dataset_None['SNR_E_P'] < 20)].copy()

dataset_None_snr15 = dataset_None[(dataset_None['SNR_Z_P'] >= 15) & (dataset_None['SNR_N_P'] >= 15) &
                                  (dataset_None['SNR_E_P'] >= 15) &
                                  (dataset_None['SNR_Z_P'] < 18) &
                                  (dataset_None['SNR_N_P'] < 18) &
                                  (dataset_None['SNR_E_P'] < 18)].copy()

dataset_None_snr10 = dataset_None[(dataset_None['SNR_Z_P'] >= 10) & (dataset_None['SNR_N_P'] >= 10) &
                                  (dataset_None['SNR_E_P'] >= 10) & (dataset_None['SNR_Z_P'] < 15) &
                                  (dataset_None['SNR_N_P'] < 15) & (dataset_None['SNR_E_P'] < 15)].copy()

dataset_None_snr5 = dataset_None[(dataset_None['SNR_Z_P'] >= 5) & (dataset_None['SNR_N_P'] >= 5) &
                                 (dataset_None['SNR_E_P'] >= 5) & (dataset_None['SNR_Z_P'] < 10) &
                                 (dataset_None['SNR_N_P'] < 10) & (dataset_None['SNR_E_P'] < 10)].copy()

dataset_None_snr0 = dataset_None[(dataset_None['SNR_Z_P'] > 0) & (dataset_None['SNR_N_P'] > 0) &
                                 (dataset_None['SNR_E_P'] > 0) & (dataset_None['SNR_Z_P'] < 5) &
                                 (dataset_None['SNR_N_P'] < 5) & (dataset_None['SNR_E_P'] < 5)].copy()

# Filtrar los valores que cumplan alguna de las condiciones anteriores o tengan algún valor None en las columnas
dataset_None_invalid = dataset_None[~(
    (dataset_None['SNR_Z_P'] >= 25) & (dataset_None['SNR_N_P'] >= 25) & (dataset_None['SNR_E_P'] >= 25) |
    ((dataset_None['SNR_Z_P'] >= 20) & (dataset_None['SNR_N_P'] >= 20) & (dataset_None['SNR_E_P'] >= 20) &
    (dataset_None['SNR_Z_P'] < 25) & (dataset_None['SNR_N_P'] < 25) & (dataset_None['SNR_E_P'] < 25)) |
    ((dataset_None['SNR_Z_P'] >= 18) & (dataset_None['SNR_N_P'] >= 18) & (dataset_None['SNR_E_P'] >= 18) &
    (dataset_None['SNR_Z_P'] < 20) & (dataset_None['SNR_N_P'] < 20) & (dataset_None['SNR_E_P'] < 20)) |    
    ((dataset_None['SNR_Z_P'] >= 15) & (dataset_None['SNR_N_P'] >= 15) & (dataset_None['SNR_E_P'] >= 15) &
    (dataset_None['SNR_Z_P'] < 18) & (dataset_None['SNR_N_P'] < 18) & (dataset_None['SNR_E_P'] < 18)) |
    ((dataset_None['SNR_Z_P'] >= 10) & (dataset_None['SNR_N_P'] >= 10) & (dataset_None['SNR_E_P'] >= 10) &
    (dataset_None['SNR_Z_P'] < 15) & (dataset_None['SNR_N_P'] < 15) & (dataset_None['SNR_E_P'] < 15)) |
    ((dataset_None['SNR_Z_P'] >= 5) & (dataset_None['SNR_N_P'] >= 5) & (dataset_None['SNR_E_P'] >= 5) &
    (dataset_None['SNR_Z_P'] < 10) & (dataset_None['SNR_N_P'] < 10) & (dataset_None['SNR_E_P'] < 10)) |
    ((dataset_None['SNR_Z_P'] > 0) & (dataset_None['SNR_N_P'] > 0) & (dataset_None['SNR_E_P'] > 0) &
    (dataset_None['SNR_Z_P'] < 5) & (dataset_None['SNR_N_P'] < 5) & (dataset_None['SNR_E_P'] < 5)))].copy()



# Marcar la columna 'Is_Valid' en cada DataFrame
dataset_None_snr25['Is_Valid'] = True
dataset_None_snr20['Is_Valid'] = False
dataset_None_snr18['Is_Valid'] = False
dataset_None_snr15['Is_Valid'] = False
dataset_None_snr10['Is_Valid'] = False
dataset_None_snr5['Is_Valid'] = False
dataset_None_snr0['Is_Valid'] = False
dataset_None_invalid['Is_Valid'] = False

# Verificar los nuevos DataFrames
print("Dataset None SNR >= 25:")
print(dataset_None_snr25.shape)

print("\nDataset None SNR >= 20:")
print(dataset_None_snr20.shape)

print("\nDataset None SNR >= 18:")
print(dataset_None_snr18.shape)

print("\nDataset None SNR >= 15:")
print(dataset_None_snr15.shape)

print("\nDataset NoneSNR >= 10:")
print(dataset_None_snr10.shape)

print("\nDataset None SNR >= 5:")
print(dataset_None_snr5.shape)

print("\nDataset None SNR > 0:")
print(dataset_None_snr0.shape)

print("\nDataset None Invalid:")
print(dataset_None_invalid.shape)



In [ ]:
#Restablezco los indices del dataset eliminando las discontinuidades por las filas eliminadas
dataset_None_snr25 = dataset_None_snr25.reset_index(drop=True)
dataset_None_snr20 = dataset_None_snr20.reset_index(drop=True)
dataset_None_snr18 = dataset_None_snr18.reset_index(drop=True)
dataset_None_snr15 = dataset_None_snr15.reset_index(drop=True)
dataset_None_snr10 = dataset_None_snr10.reset_index(drop=True)
dataset_None_snr5 = dataset_None_snr5.reset_index(drop=True)
dataset_None_snr0 = dataset_None_snr0.reset_index(drop=True)
dataset_None_invalid = dataset_None_invalid.reset_index(drop=True)


In [ ]:
dataset_None_snr_any_channel = dataset_None_invalid[((dataset_None_invalid['SNR_Z_P'] >= 15) &
                                                     ((dataset_None_invalid['SNR_N_P'] >= 10) &
                                                     (dataset_None_invalid['SNR_E_P'] >= 10)))|
                                                     ((dataset_None_invalid['SNR_N_P'] >= 15) &
                                                     ((dataset_None_invalid['SNR_Z_P'] >= 10) &
                                                     (dataset_None_invalid['SNR_E_P'] >= 10)))|
                                                     ((dataset_None_invalid['SNR_E_P'] >= 15) &
                                                     ((dataset_None_invalid['SNR_Z_P'] >= 10) &
                                                     (dataset_None_invalid['SNR_N_P'] >= 10)))].copy()

dataset_None_new_invalid = dataset_None_invalid[~(((dataset_None_invalid['SNR_Z_P'] >= 15) &
                                                  ((dataset_None_invalid['SNR_N_P'] >= 10) &
                                                  (dataset_None_invalid['SNR_E_P'] >= 10)))| 
                                                  ((dataset_None_invalid['SNR_N_P'] >= 15) &
                                                  ((dataset_None_invalid['SNR_Z_P'] >= 10) &
                                                  (dataset_None_invalid['SNR_E_P'] >= 10)))| 
                                                  ((dataset_None_invalid['SNR_E_P'] >= 15) &
                                                  ((dataset_None_invalid['SNR_Z_P'] >= 10) &
                                                  (dataset_None_invalid['SNR_N_P'] >= 10))))].copy()





#Restablezco los indices del dataset eliminando las discontinuidades por las filas eliminadas
dataset_None_snr_any_channel = dataset_None_snr_any_channel.reset_index(drop=True)
dataset_None_new_invalid = dataset_None_new_invalid.reset_index(drop=True)
# Verificar los nuevos DataFrames
print("Dataset SNR None Any Channel >= 15:")
print(dataset_None_snr_any_channel.shape)

print("Dataset SNR None New Invalid:")
print(dataset_None_new_invalid.shape)


In [ ]:
# Salvo los datasets
events_container_path= "C:\\TESIS\\FINAL\\Training\\Preprocesing\\SNR_Datasets_validos\\" 
dataset.to_pickle(events_container_path + 'dataset.pkl')
dataset_snr25.to_pickle(events_container_path + 'dataset_snr25.pkl') 
dataset_snr20.to_pickle(events_container_path + 'dataset_snr20.pkl') 
dataset_snr18.to_pickle(events_container_path + 'dataset_snr18.pkl') 
dataset_snr15.to_pickle(events_container_path + 'dataset_snr15.pkl') 
dataset_snr10.to_pickle(events_container_path + 'dataset_snr10.pkl') 
dataset_snr5.to_pickle(events_container_path + 'dataset_snr5.pkl') 
dataset_snr0.to_pickle(events_container_path + 'dataset_snr0.pkl') 
dataset_invalid.to_pickle(events_container_path + 'dataset_invalid.pkl')
dataset_snr_any_channel.to_pickle(events_container_path + 'dataset_snr_any_channel.pkl')
dataset_new_invalid.to_pickle(events_container_path + 'dataset_new_invalid.pkl')

dataset_None_snr25.to_pickle(events_container_path + 'dataset_None_snr25.pkl') 
dataset_None_snr20.to_pickle(events_container_path + 'dataset_None_snr20.pkl') 
dataset_None_snr18.to_pickle(events_container_path + 'dataset_None_snr18.pkl') 
dataset_None_snr15.to_pickle(events_container_path + 'dataset_None_snr15.pkl') 
dataset_None_snr10.to_pickle(events_container_path + 'dataset_None_snr10.pkl') 
dataset_None_snr5.to_pickle(events_container_path + 'dataset_None_snr5.pkl') 
dataset_None_snr0.to_pickle(events_container_path + 'dataset_None_snr0.pkl') 
dataset_None.to_pickle(events_container_path + 'dataset_None.pkl') 
dataset_None_invalid.to_pickle(events_container_path + 'dataset_None_invalid.pkl') 
dataset_None_snr_any_channel.to_pickle(events_container_path + 'dataset_None_snr_any_channel.pkl')
dataset_None_new_invalid.to_pickle(events_container_path + 'dataset_None_new_invalid.pkl')


In [ ]:
# Aqui comienza la segunda notebook

In [ ]:
# Se crea el dataset final a partir de los datasets anteriores

In [ ]:
# Se mezcla en un solo dataset
dataset_valid = pd.concat([dataset_snr25, dataset_snr20, dataset_snr18, dataset_snr15, dataset_snr_any_channel, 
                           dataset_None_snr25, dataset_None_snr20, dataset_None_snr15, dataset_None_snr10,
                           dataset_None_snr_any_channel])
dataset_valid = dataset_valid.reset_index(drop=True)
dataset_valid.shape

In [ ]:
dataset_valid

In [ ]:
#Garantizo que la columna Is_valid este en True
dataset_valid['Is_Valid'] = True

In [ ]:
# Se busca correlacion en los datos y posibles outliers

sns.pairplot(dataset_valid[['Azimuth','Distance','Mc','Depth', 'SNR_Z_S', 'SNR_N_S',
                            'SNR_E_S','SNR_Z_P', 'SNR_N_P', 'SNR_E_P']], diag_kind="kde")

In [ ]:
# Se hace un histograma de la variable a predecir "azimut"
frequencies, bins, _ = plt.hist(dataset_valid['Azimuth'], bins=20)
plt.xlabel('Azimut')
plt.ylabel('Frecuencia')

# Se calcula el valor medio de las frecuencias
mean_frequency = frequencies.mean()
plt.axhline(mean_frequency, color='r', linestyle='--', label='Frecuencia media')

plt.legend()
plt.show()

print(f'La frecuencia media de los valores de azimut es {mean_frequency:.2f}.')

In [ ]:
# Se crea un dataframe por cada azimuth
list_azimuths = []
for n in range(360): 
    df_mask = dataset_valid['Azimuth'] == n
    list_azimuths.append(dataset_valid[df_mask])

In [ ]:
# Se cuenta la cantidad de datos que hay cada azimuth e imprimo (valor del azimuth, cantidad, )
azimuth_quantity = []
for n in range(360): 
    azimuth_quantity.append(len(list_azimuths[n]))
    
#azimuth_quantity.index(max(azimuth_quantity)),max(azimuth_quantity)
azimuth_quantity
for indice, cantidad in enumerate(azimuth_quantity):
    print(f"Azimuth {indice}: {cantidad}")

In [ ]:
list_azimuths[250]

In [ ]:
#Ploteo el evento
list_azimuths[250].iloc[0]['RCC_OBSPY'].plot()

In [ ]:
container_path= "C:\\TESIS\\data_preprocessing\\SNR_Datasets_final\\"
dataset_valid.to_pickle(container_path +'dataset_valid.pkl')


In [ ]:
# Aqui comienza la 3ra notebook

In [ ]:
#Codigo para crear , los datos de entrada X1, X2 y de salida y1=cos(azimuth) , y2 = sin(azimuth)

In [ ]:
# Se importa el conjunto de datos 
dataset = dataset_valid.copy()
dataset.shape

In [ ]:
dataset

In [ ]:
# Se hace un histograma de la variable a predecir "azimut en el DataSet "
frequencies, bins, _ = plt.hist(dataset['Azimuth'], bins=300)
plt.xlabel('Azimut')
plt.ylabel('Frecuencia')

# Se calcula el valor medio de las frecuencias
mean_frequency = frequencies.mean()
plt.axhline(mean_frequency, color='r', linestyle='--', label='Frecuencia media')
#plt.title('Distribución de la variable a predecir en el Conjunto de Entrenamiento')
plt.legend()
plt.show()
print(f'La frecuencia media de los valores de azimut es {mean_frequency:.2f}.')

In [ ]:
#Me quedo con los azimuth en el rango 90- 270

In [ ]:
# Elimino los eventos con Azimuth 0 a 90

indexNames = dataset[ ((dataset['Azimuth'] < 90))].index
dataset.drop(indexNames , inplace=True)
#sns.pairplot(dataset[['Azimuth','Distance','Mc','Depth']], diag_kind="kde")
dataset.shape

In [ ]:
# Elimino los eventos con Azimuth 270 a 359

indexNames = dataset[ ((dataset['Azimuth'] > 270))].index
dataset.drop(indexNames , inplace=True)
#sns.pairplot(dataset[['Azimuth','Distance','Mc','Depth']], diag_kind="kde")
dataset.shape

In [ ]:
# Elimino los Azimuts de los que se tiene menos de dos muestras
class_counts = dataset["Azimuth"].value_counts()
classes_to_remove = class_counts[class_counts < 2].index
filtered_dataset = dataset[~dataset["Azimuth"].isin(classes_to_remove)]

In [ ]:
filtered_dataset.shape

In [ ]:
# Definir las variables para la estratificación
features = ["Azimuth", "Mc", "Distance", "Depth"]

# Dividir el dataframe en características (X) y variable objetivo (y)
X = filtered_dataset.drop("Azimuth", axis=1)
y = filtered_dataset["Azimuth"]

# Realizar la estratificación utilizando StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_index, test_index = next(sss.split(X,y))

# Obtener conjuntos de entrenamiento y prueba utilizando los índices generados
X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
dataset_train_merged = X_train.join(y_train)

In [ ]:
dataset_test_merged = X_test.join(y_test)

In [ ]:
dataset_train_merged

In [ ]:
dataset_test_merged

In [ ]:
#Restablezco los indices del dataset eliminando las discontinuidades por las filas eliminadas
dataset_test = dataset_test_merged.reset_index(drop=True)
dataset_test

In [ ]:
#Restablezco los indices del dataset eliminando las discontinuidades por las filas eliminadas
dataset_train = dataset_train_merged.reset_index(drop=True)
dataset_train

In [ ]:
# Se hace un histograma de la variable a predecir "azimut en el Set de Training"
frequencies, bins, _ = plt.hist(dataset_train['Azimuth'], bins=280)
plt.xlabel('Azimut')
plt.ylabel('Frecuencia')

# Se calcula el valor medio de las frecuencias
mean_frequency = frequencies.mean()
plt.axhline(mean_frequency, color='r', linestyle='--', label='Frecuencia media')
plt.title('Distribución de la variable a predecir en el Conjunto de Entrenamiento')
plt.legend()
plt.show()
print(f'La frecuencia media de los valores de azimut es {mean_frequency:.2f}.')

In [ ]:
# Se hace un histograma de la variable a predecir "azimut en el set de Test"
frequencies, bins, _ = plt.hist(dataset_test['Azimuth'], bins=280)
plt.xlabel('Azimut')
plt.ylabel('Frecuencia')

# Se calcula el valor medio de las frecuencias
mean_frequency = frequencies.mean()
plt.axhline(mean_frequency, color='r', linestyle='--', label='Frecuencia media')
plt.title('Distribución de la variable a predecir en el Conjunto de Validacion')
plt.legend()
plt.show()

print(f'La frecuencia media de los valores de azimut es {mean_frequency:.2f}.')

In [ ]:
# Se muestra correlacion en los datos en el dataset de Train
sns.pairplot(dataset_train[['Azimuth','Distance','Mc','Depth', ]], diag_kind="kde")

In [ ]:
#Para conocer la composicion del dataset de Validacion

In [ ]:
# Se crea un dataframe por cada azimuth
list_azimuths_test = []
for n in range(360): 
    df_mask_test = dataset_test['Azimuth'] == n
    list_azimuths_test.append(dataset_test[df_mask_test])

In [ ]:
# Se cuenta la cantidad de datos que hay cada azimuth e imprimo (valor del azimuth, cantidad, )
azimuth_quantity_test = []
for n in range(360): 
    azimuth_quantity_test.append(len(list_azimuths_test[n]))
#azimuth_quantity.index(max(azimuth_quantity)),max(azimuth_quantity)
for indice, cantidad in enumerate(azimuth_quantity_test):
    print(f"Azimuth {indice}: {cantidad}")

In [ ]:
# Se crea un dataframe por cada azimuth del dataset Train
list_azimuths_train = []
for n in range(360): 
    df_mask_train = dataset_train['Azimuth'] == n
    list_azimuths_train.append(dataset_train[df_mask_train])

In [ ]:
# Se cuenta la cantidad de datos que hay cada azimuth e imprimo (valor del azimuth, cantidad, )
azimuth_quantity_train = []
for n in range(360): 
    azimuth_quantity_train.append(len(list_azimuths_train[n]))
for indice, cantidad in enumerate(azimuth_quantity_train):
    print(f"Azimuth {indice}: {cantidad}")

In [ ]:
# Se conforman el vector de entrada de Training X1_train, 
#Se escoge la ventana de tiempo a utilizar, 
#Se forma la entrada2 Matriz de Covarianza, Vector Propio y Valor Propio 

window_width=500
samples_number=len(dataset_train)
X1_train=np.empty(shape=[0,window_width,3])
for n in range(samples_number): 
    print('Procesando :',n)
    tmp=dataset_train.iloc[n]
    z_original= tmp['Z_data']    
    n_original= tmp['N_data']    
    e_original= tmp['E_data'] 
    z_max = np.max(z_original)
    n_max = np.max(n_original)
    e_max = np.max(e_original)
    print('z_max :',z_max)
    print('n_max :',n_max)
    print('e_max :',e_max)
    global_max = z_max
    if global_max < n_max :
        global_max = n_max
    if global_max < e_max :
        global_max = e_max
    print('global_max :',global_max)
    z_norm = z_original/global_max
    n_norm = n_original/global_max
    e_norm = e_original/global_max    
    new_z= z_norm[0: window_width]    
    new_n= n_norm[0: window_width]
    new_e= e_norm[0: window_width]    
    zne=np.dstack((new_z,new_n,new_e)) 
    X1_train=np.append(X1_train,zne, axis=0)     
print(X1_train.shape)

In [ ]:
# Se conforman los vectores de entrada de Test X1_test y X2_test, 
#Se escoge la ventana de tiempo a utilizar, 
#Se forma la entrada2 Matriz de Covarianza, Vector Propio y Valor Propio 

window_width=500
samples_number=len(dataset_test)
X1_test=np.empty(shape=[0,window_width,3])
for n in range(samples_number): 
    print('Procesando :',n)
    tmp=dataset_test.iloc[n]
    z_original= tmp['Z_data']    
    n_original= tmp['N_data']    
    e_original= tmp['E_data'] 
    z_max = np.max(z_original)
    n_max = np.max(n_original)
    e_max = np.max(e_original)
    print('z_max :',z_max)
    print('n_max :',n_max)
    print('e_max :',e_max)
    global_max = z_max
    if global_max < n_max :
        global_max = n_max
    if global_max < e_max :
        global_max = e_max
    print('global_max :',global_max)
    z_norm = z_original/global_max
    n_norm = n_original/global_max
    e_norm = e_original/global_max    
    new_z= z_norm[0: window_width]    
    new_n= n_norm[0: window_width]
    new_e= e_norm[0: window_width]    
    zne=np.dstack((new_z,new_n,new_e)) 
    X1_test=np.append(X1_test,zne, axis=0)    
print(X1_test.shape)

In [ ]:
# Se conforman los vectores de salida de Training, 

samples_number=len(dataset_train)
y_train_cos=np.empty(shape=[samples_number])
y_train_sin=np.empty(shape=[samples_number])
for n in range(samples_number):   
    tmp=dataset_train.iloc[n]    
    
    azimuth= tmp['Azimuth']
    if azimuth !=0 and azimuth !=90 and azimuth !=180 and azimuth !=270 :
        y_train_cos[n]= math.cos(math.radians(azimuth))
        y_train_sin[n]= math.sin(math.radians(azimuth))
    else:
        if azimuth == 0:
            y_train_cos[n]=1
            y_train_sin[n]=0
        if azimuth == 90:
            y_train_cos[n]=0
            y_train_sin[n]=1
        if azimuth == 180:
            y_train_cos[n]=-1
            y_train_sin[n]=0
        if azimuth == 270:
            y_train_cos[n]=0
            y_train_sin[n]=-1    
print('y_train_cos',y_train_cos)
print('y_train_sin',y_train_sin)     

In [ ]:
plt.scatter(y_train_cos,y_train_sin )
plt.axis('square')
plt.xlabel('Cos(azimut)')
plt.ylabel('Sen(azimuth)')

In [ ]:
# Se conforman los vectores de salida de Test Version 

samples_number=len(dataset_test)
y_test_cos=np.empty(shape=[samples_number])
y_test_sin=np.empty(shape=[samples_number])
for n in range(samples_number):   
    tmp=dataset_test.iloc[n]
    azimuth= tmp['Azimuth']
    if azimuth !=0 and azimuth !=90 and azimuth !=180 and azimuth !=270 :
        y_test_cos[n]= math.cos(math.radians(azimuth))
        y_test_sin[n]= math.sin(math.radians(azimuth))
    else:
        if azimuth == 0:
            y_test_cos[n]=1
            y_test_sin[n]=0
        if azimuth == 90:
            y_test_cos[n]=0
            y_test_sin[n]=1
        if azimuth == 180:
            y_test_cos[n]=-1
            y_test_sin[n]=0
        if azimuth == 270:
            y_test_cos[n]=0
            y_test_sin[n]=-1    
print('y_test_cos',y_test_cos)
print('y_test_sin',y_test_sin)


In [ ]:
plt.scatter(y_test_cos, y_test_sin)
plt.axis('square')
plt.xlabel('Cos(azimut)')
plt.ylabel('Sen(azimuth)')

In [ ]:
# Salvo los vectores de entrada  y salida de Training
container_path= "C:\\TESIS\\FINAL\\Training\\Preprocesing\\Input_Output_Vectors\\"
np.save(container_path + 'X1_train.npy',X1_train)
np.save(container_path + 'y_train_cos.npy',y_train_cos)
np.save(container_path + 'y_train_sin.npy',y_train_sin)

In [ ]:
# Salvo los vectores de entrada  y salida de Test
np.save(container_path + 'X1_test.npy',X1_test)
np.save(container_path + 'y_test_cos.npy',y_test_cos)
np.save(container_path + 'y_test_sin.npy',y_test_sin)

In [ ]:
# Salvo los dataset de Training y de Test
dataset_train.to_pickle(container_path +'dataset_train.pkl')
dataset_test.to_pickle(container_path +'dataset_test.pkl')
